# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,14.87,81,65,3.13,PT,1732941848
1,1,tucupita,9.0622,-62.0510,23.05,99,5,1.30,VE,1732941849
2,2,waitangi,-43.9535,-176.5597,11.57,74,15,3.13,NZ,1732941671
3,3,zakamensk,50.3741,103.2863,-4.51,57,100,1.20,RU,1732941851
4,4,mareeba,-17.0000,145.4333,27.35,68,100,2.72,AU,1732941852


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot, parametes and sizing

map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity',
    color='City', 
    alpha=0.5,
    tiles='OSM',
    title='City Humidity Map'
).opts(
    width=1100,  
    height=600  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] > 21.00) & 
							 (city_data_df['Max Temp'] < 27.00) &
							 (city_data_df['Wind Speed'] < 4.50) &
							 (city_data_df['Cloudiness'] == 0), :]

# Drop any rows with null values

ideal_city_df = ideal_city_df.dropna(how='any')

# Display sample data

ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,churu,28.3000,74.9500,23.13,17,0,1.64,IN,1732941911
116,116,rampur,28.8167,79.0333,22.11,33,0,2.56,IN,1732941987
129,129,brisas de zicatela,15.8369,-97.0419,24.29,88,0,3.09,MX,1732942003
145,145,manta,-0.9500,-80.7333,22.88,83,0,3.09,EC,1732942022
220,220,acapulco de juarez,16.8634,-99.8901,25.90,89,0,1.03,MX,1732942086
236,236,sibi,29.5448,67.8764,24.18,26,0,0.48,PK,1732941843
360,360,bonthe,7.5264,-12.5050,24.61,87,0,1.95,SL,1732942285
391,391,tazacorte,28.6290,-17.9293,22.30,34,0,4.02,ES,1732942323
406,406,boa vista,2.8197,-60.6733,26.99,74,0,2.57,BR,1732942205


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ""

# Display sample data

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
53,churu,IN,28.3000,74.9500,17,
116,rampur,IN,28.8167,79.0333,33,
129,brisas de zicatela,MX,15.8369,-97.0419,88,
145,manta,EC,-0.9500,-80.7333,83,
220,acapulco de juarez,MX,16.8634,-99.8901,89,
236,sibi,PK,29.5448,67.8764,26,
360,bonthe,SL,7.5264,-12.5050,87,
391,tazacorte,ES,28.6290,-17.9293,34,
406,boa vista,BR,2.8197,-60.6733,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'limit' : 1,
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    
    citylat = row['Lat']
    citylon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{citylon},{citylat},{radius}"
    params["bias"] =  f"proximity:{citylon},{citylat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
churu - nearest hotel: No hotel found
rampur - nearest hotel: تمنا ہوٹل اور ریستوران
brisas de zicatela - nearest hotel: Casa de Olas
manta - nearest hotel: Hotel Manta Real
acapulco de juarez - nearest hotel: Hotel del Valle
sibi - nearest hotel: Baba Hotel & Restaurant (بابا ھوٹل ریسٹورنٹ)
bonthe - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
boa vista - nearest hotel: Aipana Plaza Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
53,churu,IN,28.3000,74.9500,17,No hotel found
116,rampur,IN,28.8167,79.0333,33,تمنا ہوٹل اور ریستوران
129,brisas de zicatela,MX,15.8369,-97.0419,88,Casa de Olas
145,manta,EC,-0.9500,-80.7333,83,Hotel Manta Real
220,acapulco de juarez,MX,16.8634,-99.8901,89,Hotel del Valle
236,sibi,PK,29.5448,67.8764,26,Baba Hotel & Restaurant (بابا ھوٹل ریسٹورنٹ)
360,bonthe,SL,7.5264,-12.5050,87,No hotel found
391,tazacorte,ES,28.6290,-17.9293,34,App Leyma
406,boa vista,BR,2.8197,-60.6733,74,Aipana Plaza Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot

ideal_map_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity',
    color='City', 
    alpha=0.5,
    tiles='OSM',
    title='City Humidity Map'
).opts(
    width=1100,  
    height=600  
)

# Display the map

ideal_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)